In [1]:
import numpy as np
import pandas as pd
import random
import copy
import networkx as nx
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

In [2]:
def game_community_opinion(W, S_RANDOM, X_INI, random_choose, Lambda = 1.5, BETA = 1, gamma = 0.95, ETA=0.1):
    """
    ################################ 输入参数 ###############################
        Influence matrix W : n*n, 
        initial opinion vector X_INI : n*1, 
        equilibrium initial community membership matrix S_INI : n*c,
        diagonal matrix Lambda_MATRIX : n*n,
        tuning parameters BETA and gamma.
    """
    t = 0
    S_STAR = S_RANDOM
    row_identity = np.ones((n, 1))
    ALL_ONES = np.ones((n, n))
    
    Lambda_MATRIX = Lambda*np.eye(n)
    
    ############### 节点i的邻居节点 ： W_ij + W_ji > 0 and i #####################
    Neighbor_set = []
    Neighbor_MATRIX = W + W.T + np.eye(n)
    for i in range(n):
        Neighbor_set.append(list(np.where(Neighbor_MATRIX[i])[0]))
        
    FINAL_PHI_S = 0
    
    while 1:
        #print('########################### t = %d ######################'%t)
        
        E = X_INI*row_identity.T - row_identity*X_INI.T
        E = np.abs(E)
        D = ETA - E
        B = np.maximum(0, D)
        
        C = np.matmul(Lambda_MATRIX, D) - B
        Y = C*W+C.T*W.T
        
        Z = Y.reshape(n, 1, n)
        
        S_INI = copy.deepcopy(S_STAR)
        S_Y = np.matmul(S_INI.T, Y)
        S_Y_S = np.matmul(S_Y, S_INI)
        PHI_S = 0.5*S_Y_S.trace()
        
        DELTA_PHI = 1e10
        
        ################################  更新社区标签  ################################
        while DELTA_PHI!=0:
            DELTA_PHI = 0
            for i in range(n):
                F_i = S_INI[Neighbor_set[i]]
                S_S = np.matmul(F_i, S_INI.T)
                P_i = np.matmul(S_S, Z[i].T)
                
                # V_i = P_i.argmax()
                if np.random.rand() < random_choose:
                    # 随机选择一个邻居索引
                    V_i = np.random.randint(0, len(P_i))
                else:
                    # 正常贪心选择
                    V_i = int(np.argmax(P_i))
                
                S_i_star = F_i[V_i]

                S_MINUS_S_S = np.matmul(S_i_star - S_INI[i], S_INI.T)
                DELTA_PHI += np.matmul(S_MINUS_S_S, Z[i].T)
                S_INI[i] = S_i_star
                            
            PHI_S += DELTA_PHI[0]
            #print("PHI_S : %d"%PHI_S)
            
        beta_gamma = BETA*pow(gamma, t)
        Xi_t = beta_gamma*ALL_ONES
        relu_D = np.maximum(0, Xi_t - E)
        W_D = W*relu_D
        
        X_S_S = np.matmul(S_INI, S_INI.T)
        Lambda_MODIFY = Lambda*ALL_ONES - 1
        X_S_S_MODIFY = 1 + Lambda_MODIFY*X_S_S
        
        THETA = X_S_S_MODIFY*W_D
        
        GAMMA = np.matmul(THETA, ALL_ONES) + ALL_ONES
        PHI = THETA / GAMMA
        
        PHI_prime = np.matmul(THETA, row_identity) + row_identity
        PHI_prime = row_identity / PHI_prime
        
        X_UPDATE = PHI_prime*X_INI + np.matmul(PHI, X_INI)
        
        S_STAR = S_INI
        
#         print("PHI_S : ",PHI_S)
#         print("FINAL_PHI_S : ",FINAL_PHI_S)
        
        DETA_X = np.abs(X_UPDATE - X_INI)
        if DETA_X.max() < 1e-3:
            break
        
        X_INI = X_UPDATE
        FINAL_PHI_S = PHI_S
        
        t += 1
    
    return S_STAR, X_UPDATE, PHI

In [3]:
"""
##############################  全局变量  #############################
n : 节点数量
c : 初始社区数量
"""
all_edges = pd.read_csv('PolBlogs.edges', header=None, sep='\t')
all_edges = all_edges - 1
row = list(all_edges.iloc[:, 0])
col = list(all_edges.iloc[:, 1])
print("边数：", len(row))

W = np.zeros((793, 793))
for i in range(len(row)):
    W[row[i]][col[i]]=1

n = W.shape[0]
ALL_ONES = np.ones((n, n))

G_COMPUTE_Q = nx.from_numpy_array(W)

all_community = pd.read_csv('PolBlogs.gt', header=None)
TRUE_COMMUNITY = np.zeros(n)
all_nodes_number = 0
for i in range(len(all_community.iloc[:, 0])):
    community_node = all_community.iloc[:, 0][i].split(' ')[:-1]
    all_nodes_number += len(community_node)
    nodes_label = [int(community_node[i]) for i in range(len(community_node))]
    nodes_label = np.array(nodes_label) - 1
    TRUE_COMMUNITY[nodes_label] = i

c = 20

边数： 15783


In [ ]:
######################### 给定初始社区结构
S = np.zeros((n, c))
random.seed(2)
community_index = [random.randint(0, c-1) for _ in range(n)]
for i in range(n):
    S[i][community_index[i]]=1

random_o_all_ami = []
random_o_all_ari = []

for random_value in [0.0, 0.01, 0.02, 0.03, 0.04, 0.05]:
    o_all_ami = []
    o_all_ari = []
    
    for x_seed in range(100):
        np.random.seed(x_seed)
        X = np.random.uniform(0, 1, [n, 1])
            
        FINAL_S, FINAL_X, FINAL_WEIGHT = game_community_opinion(W, S, X, random_value, Lambda = 1.2, ETA = 0.2, gamma = 0.9)
    
        NODES_LABELS = np.where(FINAL_S==1)[1]
        COMMUNITY_LABELS = list(set(NODES_LABELS))
        
        COMMUNITY_PROPAGATION = []
        for i in range(len(COMMUNITY_LABELS)):
            COMMUNITY_LABEL_SET = []
            COMMUNITY_LABEL_ONE = COMMUNITY_LABELS[i]
            for j in range(n):
                if NODES_LABELS[j]==COMMUNITY_LABEL_ONE:
                    COMMUNITY_LABEL_SET.append(j)
            COMMUNITY_PROPAGATION.append(COMMUNITY_LABEL_SET)
    
        ari = adjusted_rand_score(TRUE_COMMUNITY, NODES_LABELS)
        ami = adjusted_mutual_info_score(TRUE_COMMUNITY, NODES_LABELS)
        o_all_ari.append(ari)
        o_all_ami.append(ami)
    
    print("ari mean: ", np.array(o_all_ari).mean())
    print("ami mean: ", np.array(o_all_ami).mean())

    random_o_all_ami.append(o_all_ami)
    random_o_all_ari.append(o_all_ari)

ari mean:  0.8033711038382103
ami mean:  0.727917217355098


In [ ]:
######################### 给定初始意图向量
np.random.seed(1)
X = np.random.uniform(0, 1, [n, 1])

random_c_all_ami = []
random_c_all_ari = []

for random_value in [0.0, 0.01, 0.02, 0.03, 0.04, 0.05]:
    c_all_ami = []
    c_all_ari = []
    for s_seed in range(100):
    
        S = np.zeros((n, 30))
        random.seed(s_seed)
        community_index = [random.randint(0, c-1) for _ in range(n)]
        for i in range(n):
            S[i][community_index[i]]=1
    
        FINAL_S, FINAL_X, FINAL_WEIGHT = game_community_opinion(W, S, X, random_value, Lambda = 1.2, ETA = 0.2, gamma = 0.9)
    
        NODES_LABELS = np.where(FINAL_S==1)[1]
        COMMUNITY_LABELS = list(set(NODES_LABELS))
        
        COMMUNITY_PROPAGATION = []
        for i in range(len(COMMUNITY_LABELS)):
            COMMUNITY_LABEL_SET = []
            COMMUNITY_LABEL_ONE = COMMUNITY_LABELS[i]
            for j in range(n):
                if NODES_LABELS[j]==COMMUNITY_LABEL_ONE:
                    COMMUNITY_LABEL_SET.append(j)
            COMMUNITY_PROPAGATION.append(COMMUNITY_LABEL_SET)
        
        ari = adjusted_rand_score(TRUE_COMMUNITY, NODES_LABELS)
        ami = adjusted_mutual_info_score(TRUE_COMMUNITY, NODES_LABELS)
            
        c_all_ari.append(ari)
        c_all_ami.append(ami)
    
    print("ari mean: ", np.array(c_all_ari).mean())
    print("ami mean: ", np.array(c_all_ami).mean())
    random_c_all_ami.append(c_all_ami)
    random_c_all_ari.append(c_all_ari)